In [3]:
import pandas as pd

df = pd.read_json(f"data_getter/Go.json")

In [4]:
df

,code,before_change,committer_date
0,\t\tprivate HandshakeMessage createServerHands...,False,2015-03-06 10:34:14-05:00
1,\t\tprivate HandshakeMessage createServerHands...,True,2015-03-06 10:34:14-05:00
2,\t\tprivate HandshakeMessage createClientHands...,True,2015-03-06 10:34:14-05:00
3,\t\tprivate HandshakeMessage createClientHands...,False,2015-03-06 10:34:14-05:00
4,\t\tprivate static CipherSuiteCollection GetSs...,False,2015-03-06 10:35:27-05:00
...,...,...,...
6278,func (l *lookedUpGoFunction) Call(ctx context....,False,2024-04-30 14:34:59-07:00
6279,func (l *lookedUpGoFunction) CallWithStack(ctx...,False,2024-04-30 14:34:59-07:00
6280,func (l *lookedUpGoFunction) Definition() api....,False,2024-04-30 14:34:59-07:00
6281,func (m *ModuleInstance) LookupFunction(t *Tab...,False,2024-04-30 14:34:59-07:00


In [5]:
df = df.drop_duplicates(subset=['code'], ignore_index=True)
df = df.rename(columns={'before_change': 'label', 'code': 'text'})
df.loc[df.label == 'False', 'label'] = 0
df.loc[df.label == 'True', 'label'] = 1
df

,text,label,committer_date
0,\t\tprivate HandshakeMessage createServerHands...,0,2015-03-06 10:34:14-05:00
1,\t\tprivate HandshakeMessage createServerHands...,1,2015-03-06 10:34:14-05:00
2,\t\tprivate HandshakeMessage createClientHands...,1,2015-03-06 10:34:14-05:00
3,\t\tprivate HandshakeMessage createClientHands...,0,2015-03-06 10:34:14-05:00
4,\t\tprivate static CipherSuiteCollection GetSs...,0,2015-03-06 10:35:27-05:00
...,...,...,...
5917,func (l *lookedUpGoFunction) Call(ctx context....,0,2024-04-30 14:34:59-07:00
5918,func (l *lookedUpGoFunction) CallWithStack(ctx...,0,2024-04-30 14:34:59-07:00
5919,func (l *lookedUpGoFunction) Definition() api....,0,2024-04-30 14:34:59-07:00
5920,func (m *ModuleInstance) LookupFunction(t *Tab...,0,2024-04-30 14:34:59-07:00


In [6]:
print('Vulnerable:', len(df[df['label'] == 1]))
print('Not vulnerable:', len(df[df['label'] == 0]))

Vulnerable: 1691
Not vulnerable: 4231


In [8]:
# Save dataset without preprocessing ready for training
from sklearn.model_selection import train_test_split

df_nopreproc = df.drop(df.columns[[2]], axis=1)

train, test = train_test_split(df_nopreproc, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
test, validation = train_test_split(test, test_size=0.5)
train.to_json('go_nopreproc_train.json', orient='records')
validation.to_json('go_nopreproc_valid.json', orient='records')
test.to_json('go_nopreproc_test.json', orient='records')

In [9]:
# Prevent time traveling

#df['committer_date'] = pd.to_datetime(df['committer_date'])
df = df.sort_values(by='committer_date')
dfx = df.drop(df.columns[[2]], axis=1)
split_index = int(len(df) * 0.8)
train = dfx.iloc[:split_index]
test = dfx.iloc[split_index:]
test, validation = train_test_split(test, test_size=0.5)
train.to_json('go_date_train.json', orient='records')
validation.to_json('go_date_valid.json', orient='records')
test.to_json('go_date_test.json', orient='records')

In [22]:
!pwd

/lunarc/nobackup/projects/lu2024-17-13/kevin/php
